In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


# Three things that improve the output of GPT 4
1. Chain of thought prompting
2. Reflection
3. Dialoging

In [2]:

import json

import openai

client = openai.OpenAI (
    base_url = "xxxxxx",
    api_key = "xxxxxx"
)



In [6]:
# with open('constitution.json', 'r') as file:
#     constitution_data = json.load(file)
#  need to make this 5 shot
outputs =[]
def handle_query(user_query, role='user'):
    """
    Send user query to the AI endpoint based on the role (user, researcher, resolver)
    and return the AI's response.
    """
    # Adjust prompt based on role
    if role == 'chain_of_thought':
        prompt = f"{user_query} Question. \nAnswer: Let's work this out in a step by step way to be sure we have the right answer."
    elif role == 'researcher':
        prompt =  f"You are a researcher tasked with investigating the response options provided. List the flaws and faulty logic of each answer option. Let's work this out in a step by step way to be sure we have all the errors:\n{user_query}"
    elif role == 'resolver':
        prompt = f"You are a resolver tasked with 1) finding which of the response options the researcher thought was best, 2) improving that answer, and 3) Printing the improved answer in full. Let's work this out in a step by step way to be sure we have the right answer:\n{user_query} Answer:"
    # else:
    #     prompt = "Constitution: "+ role

    # Send the prompt to the AI endpoint
    completion = client.completions.create(
        model= "meta-llama/Llama-2-7b-chat-hf",
        prompt=prompt,
        max_tokens=5000

    )
    return completion.model_dump()


def handle_chain_of_thought(user_query):
    # after_context = handle_query(constitution_data)
    return handle_query(user_query, 'chain_of_thought')

def handle_researcher_analysis(options_analysis):
    return handle_query(options_analysis, 'researcher')

def handle_resolver_enhancement(best_option_analysis):
    return handle_query(best_option_analysis, 'resolver')

def handle_options(text):
  outputs.append(text['choices'][0]['text'])



def prettier(completion):
  # print(completion)
  return completion["choices"][0]['text']

# Example usage
user_query = "Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were"
# user_query = "What happens to you if you eat watermelon seeds?"
# user_query = "I left 5 clothes to dry out in the sun. It took them 5 hours to dry completely. How long would it take to dry 30 clothes?"
chain_of_thought_response = handle_chain_of_thought(user_query)
chain_of_thought_response2 = handle_chain_of_thought(user_query)
chain_of_thought_response3 = handle_chain_of_thought(user_query)
print("__________________CHAIN OF THOUGHT____________________")
handle_options((chain_of_thought_response));
handle_options((chain_of_thought_response2));
handle_options((chain_of_thought_response3));
print(prettier(chain_of_thought_response));
print(prettier(chain_of_thought_response2));
print(prettier(chain_of_thought_response3));

print((outputs))

# Assume chain_of_thought_response contains multiple options
print("______________RESEARCHER________________________")
options_analysis = outputs  # Extracted and formatted options from chain_of_thought_response
researcher_response = handle_researcher_analysis(options_analysis)
print(prettier(researcher_response))
print("___________________RESOLVER___________________")
# Assume researcher_response highlights the best option and its flaws
best_option_analysis = prettier(researcher_response)  # Extracted best option and its flaws from researcher_response
resolver_response = handle_resolver_enhancement(best_option_analysis)
print(prettier(resolver_response))


__________________CHAIN OF THOUGHT____________________

Option A: Adams.
Explanation: Davis intended to kill Adams, and it was Adams that Davis saw when he went to Adams's house. Therefore, the answer is (A) Adams.
Option B: Brooks.
Explanation: Davis mistook Brooks for Adams and shot at him. Therefore, the answer is (B) Brooks.
Option C: Case.
Explanation: Davis's shot missed Brooks but hit Case, who was nearby. Therefore, the answer is (C) Case.
Therefore, the answer is (A) Adams.

Question: Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were Question.
Step 1: Identify the person or people in the scenario who were intended to